# Diplomatura de Especialización en Desarrollo de Aplicaciones con Inteligencia Artificial - Inteligencia Artificial para Juegos (Game AI) - Sesión 6 (Ejemplo)

<font color='orange'>Entorno de Grilla MDP (Laberinto *Pathfinding Problem*) con visualizador de iteración de valor. Tarea: Completar funciones de iteración de valor.</font>

Códigos adaptados del repositorio [aima-python](https://github.com/aimacode/aima-python).

## MDP: Definition

In [ ]:
orientations = EAST, NORTH, WEST, SOUTH = [(1, 0), (0, 1), (-1, 0), (0, -1)]
turns = LEFT, RIGHT = (+1, -1)
import operator
def vector_add(a, b):
    """Component-wise addition of two vectors."""
    return tuple(map(operator.add, a, b))

def turn_heading(heading, inc, headings=orientations):
    return headings[(headings.index(heading) + inc) % len(headings)]

def turn_right(heading):
    return turn_heading(heading, RIGHT)


def turn_left(heading):
    return turn_heading(heading, LEFT)

In [ ]:
class MDP:

    """A Markov Decision Process, defined by an initial state, transition model,
    and reward function. We also keep track of a gamma value, for use by
    algorithms. The transition model is represented somewhat differently from
    the text. Instead of P(s' | s, a) being a probability number for each
    state/state/action triplet, we instead have T(s, a) return a
    list of (p, s') pairs. We also keep track of the possible states,
    terminal states, and actions for each state. [page 646]"""

    def __init__(self, init, actlist, terminals, transitions = {}, reward = None, states=None, gamma=.9):
        if not (0 < gamma <= 1):
            raise ValueError("An MDP must have 0 < gamma <= 1")

        if states:
            self.states = states
        else:
            ## collect states from transitions table
            self.states = self.get_states_from_transitions(transitions)
            
        
        self.init = init
        
        if isinstance(actlist, list):
            ## if actlist is a list, all states have the same actions
            self.actlist = actlist
        elif isinstance(actlist, dict):
            ## if actlist is a dict, different actions for each state
            self.actlist = actlist
        
        self.terminals = terminals
        self.transitions = transitions
        if self.transitions == {}:
            print("Warning: Transition table is empty.")
        self.gamma = gamma
        if reward:
            self.reward = reward
        else:
            self.reward = {s : 0 for s in self.states}
        #self.check_consistency()

    def R(self, state):
        """Return a numeric reward for this state."""
        return self.reward[state]

    def T(self, state, action):
        """Transition model. From a state and an action, return a list
        of (probability, result-state) pairs."""
        if(self.transitions == {}):
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]

    def actions(self, state):
        """Set of actions that can be performed in this state. By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""
        if state in self.terminals:
            return [None]
        else:
            return self.actlist

    def get_states_from_transitions(self, transitions):
        if isinstance(transitions, dict):
            s1 = set(transitions.keys())
            s2 = set([tr[1] for actions in transitions.values() 
                              for effects in actions.values() for tr in effects])
            return s1.union(s2)
        else:
            print('Could not retrieve states from transitions')
            return None

    def check_consistency(self):
        # check that all states in transitions are valid
        assert set(self.states) == self.get_states_from_transitions(self.transitions)
        # check that init is a valid state
        assert self.init in self.states
        # check reward for each state
        #assert set(self.reward.keys()) == set(self.states)
        assert set(self.reward.keys()) == set(self.states)
        # check that all terminals are valid states
        assert all([t in self.states for t in self.terminals])
        # check that probability distributions for all actions sum to 1
        for s1, actions in self.transitions.items():
            for a in actions.keys():
                s = 0
                for o in actions[a]:
                    s += o[0]
                assert abs(s - 1) < 0.001

The **_ _init_ _** method takes in the following parameters:

- init: the initial state.
- actlist: List of actions possible in each state.
- terminals: List of terminal states where only possible action is exit
- gamma: Discounting factor. This makes sure that delayed rewards have less value compared to immediate ones.

**R** method returns the reward for each state by using the self.reward dict.

**T** method is not implemented and is somewhat different from the text. Here we return (probability, s') pairs where s' belongs to list of possible state by taking action a in state s.

**actions** method returns list of actions possible in each state. By default it returns all actions for states other than terminal states.


## GRID MDP

Now we look at a concrete implementation that makes use of the MDP as base class. The GridMDP class in the mdp module is used to represent a grid world MDP like the one shown in  in **Fig 17.1** of the AIMA Book.

We assume for now that the environment is _fully observable_, so that the agent always knows where it is. The code should be easy to understand if you have gone through the CustomMDP example.

In [ ]:
def redondea(num, cant_decimales):
  if(num==None):
    return None
  else:
    return round(num, cant_decimales)

In [ ]:
class GridMDP(MDP):

    """A two-dimensional grid MDP, as in [Figure 17.1]. All you have to do is
    specify the grid as a list of lists of rewards; use None for an obstacle
    (unreachable state). Also, you should specify the terminal states.
    An action is an (x, y) unit vector; e.g. (1, 0) means move east."""

    def __init__(self, grid, terminals, init=(0, 0), gamma=.9):
        grid.reverse()  # because we want row 0 on bottom, not on top
        reward = {}
        states = set()
        self.rows = len(grid)
        self.cols = len(grid[0])
        self.grid = grid
        for x in range(self.cols):
            for y in range(self.rows):
                if grid[y][x] is not None:
                    states.add((x, y))
                    reward[(x, y)] = grid[y][x]
        self.states = states
        actlist = orientations
        transitions = {}
        for s in states:
            transitions[s] = {}
            for a in actlist:
                transitions[s][a] = self.calculate_T(s, a)
        MDP.__init__(self, init, actlist=actlist,
                     terminals=terminals, transitions = transitions, 
                     reward = reward, states = states, gamma=gamma)

    def calculate_T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else:
            return [(0.8, self.go(state, action)),
                    (0.1, self.go(state, turn_right(action))),
                    (0.1, self.go(state, turn_left(action)))]
    
    def T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else:
            return self.transitions[state][action]
 
    def go(self, state, direction):
        """Return the state that results from going in this direction."""
        state1 = vector_add(state, direction)
        return state1 if state1 in self.states else state

    def to_grid(self, mapping):
        """Convert a mapping from (x, y) to v into a [[..., v, ...]] grid."""
        return list(reversed([[redondea(mapping.get((x, y)),2)
                               for x in range(self.cols)]
                              for y in range(self.rows)]))

    def toGrid(self, mapping):
        """Convert a mapping from (x, y) to v into a [[..., v, ...]] grid."""
        return list(reversed([[mapping.get((x, y), None)
                               for x in range(self.cols)]
                              for y in range(self.rows)]))

    def to_arrows(self, policy):
        chars = {
            (1, 0): '>', (0, 1): '^', (-1, 0): '<', (0, -1): 'v', None: '.'}
        return self.toGrid({s: chars[a] for (s, a) in policy.items()})

The **_ _init_ _** method takes **grid** as an extra parameter compared to the MDP class. The grid is a nested list of rewards in states.

**go** method returns the state by going in particular direction by using vector_add.

**T** method is not implemented and is somewhat different from the text. Here we return (probability, s') pairs where s' belongs to list of possible state by taking action a in state s.

**actions** method returns list of actions possible in each state. By default it returns all actions for states other than terminal states.

**to_arrows** are used for representing the policy in a grid like format.

We can create a GridMDP like the one in **Fig 17.1** as follows: 

    GridMDP([[-0.04, -0.04, -0.04, +1],
            [-0.04, None,  -0.04, -1],
            [-0.04, -0.04, -0.04, -0.04]],
            terminals=[(3, 2), (3, 1)])

<img src='https://raw.githubusercontent.com/iapucp/IA-Diplomado/master/Inteligencia%20Artificial%20para%20Juegos/Codigo_Sesion6/fig171_aima.png' width=500px>
            
In fact the **sequential_decision_environment** in mdp module has been instantized using the exact same code.

In [ ]:
GridMDP([[-0.04, -0.04, -0.04, +1],
        [-0.04, None,  -0.04, -1],
        [-0.04, -0.04, -0.04, -0.04]],
        terminals=[(3, 2), (3, 1)])

In [ ]:

""" [Figure 17.1]
A 4x3 grid environment that presents the agent with a sequential decision problem.
"""

sequential_decision_environment = GridMDP([[-0.04, -0.04, -0.04, +1],
                                           [-0.04, None, -0.04, -1],
                                           [-0.04, -0.04, -0.04, -0.04]],
                                          terminals=[(3, 2), (3, 1)])


# ______________________________________________________________________________

In [ ]:
sequential_decision_environment

In [ ]:
sequential_decision_environment.grid

[[-0.04, -0.04, -0.04, -0.04],
 [-0.04, None, -0.04, -1],
 [-0.04, -0.04, -0.04, 1]]

## GRID MDP Pathfinding Problem
Markov Decision Processes can be used to find the best path through a maze. Let us consider this simple maze.
![title](https://github.com/aimacode/aima-python/blob/master/images/maze.png?raw=1)

This environment can be formulated as a GridMDP.
<br>
To make the grid matrix, we will consider the state-reward to be -0.1 for every state.
<br>
State (1, 1) will have a reward of -5 to signify that this state is to be prohibited.
<br>
State (9, 9) will have a reward of +5.
This will be the terminal state.
<br>
The matrix can be generated using the GridMDP editor or we can write it ourselves.

In [ ]:
gridXD = [
    [None, None, None, None, None, None, None, None, None, None, None], 
    [None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None, +5.0, None], 
    [None, -0.1, None, None, None, None, None, None, None, -0.1, None], 
    [None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None], 
    [None, -0.1, None, None, None, None, None, None, None, None, None], 
    [None, -0.1, None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None], 
    [None, -0.1, None, None, None, None, None, -0.1, None, -0.1, None], 
    [None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None, -0.1, None], 
    [None, None, None, None, None, -0.1, None, -0.1, None, -0.1, None], 
    [None, -5.0, -0.1, -0.1, -0.1, -0.1, None, -0.1, None, -0.1, None], 
    [None, None, None, None, None, None, None, None, None, None, None]
] 

In [ ]:
len(gridXD)

11

We have only one terminal state, (9, 9)

In [ ]:
terminals = [(9, 9)]

We define our maze environment below

In [ ]:
maze = GridMDP(gridXD, terminals)

To solve the maze, we can use the `best_policy` function along with `value_iteration`.

In [ ]:
argmax=max
def value_iteration(mdp, epsilon=0.001): epsilon: tolerancia de error.
    U1 = {s: 0 for s in mdp.states}
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    while True:
        U = U1.copy()
        delta = 0
        for s in mdp.states:
            U1[s] = R(s) + gamma * max(sum(p * U[s1] for (p, s1) in T(s, a))
                                       for a in mdp.actions(s))
            delta = max(delta, abs(U1[s] - U[s]))
        if delta <= epsilon * (1 - gamma) / gamma:
            return U

def best_policy(mdp, U):
    """Given an MDP and a utility function U, determine the best policy,
    as a mapping from state to action. (Equation 17.4)"""

    pi = {}
    for s in mdp.states:
        pi[s] = argmax(mdp.actions(s), key=lambda a: expected_utility(a, s, U, mdp))
    return pi


def expected_utility(a, s, U, mdp):
    """The expected utility of doing a in state s, according to the MDP and U."""

    return sum(p * U[s1] for (p, s1) in mdp.T(s, a))

In [ ]:
maze

In [ ]:
uu = value_iteration(maze) #resuelve las utilidades de maze
#es decir, ejecuta el value iteration 

In [ ]:
uu

{(5, 9): -0.28972320275406793,
 (4, 7): 1.3823597016890337,
 (1, 3): -0.07872276426135358,
 (6, 9): -0.37634232586787497,
 (7, 3): -0.5942537706997953,
 (9, 1): -0.8623821312777787,
 (9, 8): 4.2682926829268295,
 (7, 7): 2.5192603952355515,
 (2, 1): -0.72170380254087,
 (1, 6): 0.37914151714702293,
 (9, 4): -0.7965058489600694,
 (3, 7): 1.0918280307513455,
 (5, 1): -0.5888945373708313,
 (8, 5): -0.7324958860867973,
 (7, 2): -0.6437352855643286,
 (4, 9): -0.1910736469037466,
 (3, 3): -0.28972320275406793,
 (2, 9): 0.04923240747471591,
 (5, 5): -0.7651234106697301,
 (6, 7): 2.090082298255606,
 (6, 3): -0.525529612393064,
 (1, 5): 0.21095352724371716,
 (4, 1): -0.6390295013376613,
 (1, 1): -6.731253904720111,
 (9, 7): 3.564719694746112,
 (7, 1): -0.6871827903040341,
 (4, 5): -0.7937769294630936,
 (9, 3): -0.8213461008344954,
 (1, 4): 0.06327626780501514,
 (3, 9): -0.07872276426135359,
 (2, 3): -0.19107364690374662,
 (1, 9): 0.19495913077670843,
 (7, 5): -0.6953396714497672,
 (8, 7): 3.00804

In [ ]:
maze.to_grid(uu)

[[None, None, None, None, None, None, None, None, None, None, None],
 [None, 0.19, 0.05, -0.08, -0.19, -0.29, -0.38, -0.45, None, 5.0, None],
 [None, 0.38, None, None, None, None, None, None, None, 4.27, None],
 [None, 0.57, 0.84, 1.09, 1.38, 1.71, 2.09, 2.52, 3.01, 3.56, None],
 [None, 0.38, None, None, None, None, None, None, None, None, None],
 [None, 0.21, None, -0.82, -0.79, -0.77, -0.73, -0.7, -0.73, -0.77, None],
 [None, 0.06, None, None, None, None, None, -0.64, None, -0.8, None],
 [None, -0.08, -0.19, -0.29, -0.38, -0.46, -0.53, -0.59, None, -0.82, None],
 [None, None, None, None, None, -0.53, None, -0.64, None, -0.84, None],
 [None, -6.73, -0.72, -0.68, -0.64, -0.59, None, -0.69, None, -0.86, None],
 [None, None, None, None, None, None, None, None, None, None, None]]

In [ ]:
pi = best_policy(maze, uu)

In [ ]:
def isnumber(x):
    """Is x a number?"""
    return hasattr(x, '__int__')
    
def print_table(table, header=None, sep='   ', numfmt='{}'):
    """Print a list of lists as a table, so that columns line up nicely.
    header, if specified, will be printed as the first row.
    numfmt is the format for all numbers; you might want e.g. '{:.2f}'.
    (If you want different formats in different columns,
    don't use print_table.) sep is the separator between columns."""
    justs = ['rjust' if isnumber(x) else 'ljust' for x in table[0]]

    if header:
        table.insert(0, header)

    table = [[numfmt.format(x) if isnumber(x) else x for x in row]
             for row in table]

    sizes = list(map(lambda seq: max(map(len, seq)), list(zip(*[map(str, row) for row in table]))))

    for row in table:
        print(sep.join(getattr(str(x), j)(size) for (j, size, x) in zip(justs, sizes, row)))

Let's print out the best policy:

In [ ]:
print_table(maze.to_arrows(pi))

None   None   None   None   None   None   None   None   None   None   None
None   v      <      <      <      <      <      <      None   .      None
None   v      None   None   None   None   None   None   None   ^      None
None   >      >      >      >      >      >      >      >      ^      None
None   ^      None   None   None   None   None   None   None   None   None
None   ^      None   >      >      >      >      v      <      <      None
None   ^      None   None   None   None   None   v      None   ^      None
None   ^      <      <      <      <      <      <      None   ^      None
None   None   None   None   None   ^      None   ^      None   ^      None
None   >      >      >      >      ^      None   ^      None   ^      None
None   None   None   None   None   None   None   None   None   None   None


This is the heatmap generated by the GridMDP editor using `value_iteration` on this environment.
Se observa como la política encontrada siempre intenta llevarnos hacia el punto con mayor recompensa.
<br>
![title](https://github.com/aimacode/aima-python/blob/master/images/mdp-d.png?raw=1)
<br>

### Solving the example from Fig. 17.1

In [ ]:
sequential_decision_environment

In [ ]:
u_171 = value_iteration(sequential_decision_environment)

In [ ]:
u_171

{(0, 1): 0.3984432178350045,
 (1, 2): 0.649585681261095,
 (3, 2): 1.0,
 (0, 0): 0.2962883154554812,
 (3, 0): 0.12987274656746342,
 (3, 1): -1.0,
 (2, 1): 0.48644001739269643,
 (2, 0): 0.3447542300124158,
 (2, 2): 0.7953620878466678,
 (1, 0): 0.25386699846479516,
 (0, 2): 0.5093943765842497}

In [ ]:
pi_171 = best_policy(sequential_decision_environment, u_171)

In [ ]:
print_table(sequential_decision_environment.to_arrows(pi_171))

>   >      >   .
^   None   ^   .
^   >      ^   <


### Ejemplo más sencillo

Usando una [GUI](https://github.com/aimacode/aima-python/blob/master/gui/grid_mdp.py) disponible en el repositorio AIMA, podemos generar un entorno MDP del tipo grilla y visualizar en un *heatmap* los valores encontrados luego de las iteraciones.

In [ ]:
gridXD = [
    [None, None, None, None, None, None, None],
    [None, -0.1, -0.1, -0.1, -0.1, +5.0, None],
    [None, -0.1, None, None, None, None, None],
    [None, -0.1, -0.1, -0.1, -0.1, -0.1, None],
    [None, None, -0.1, None, -0.1, None, None],
    [None, -5.0, -0.1, -0.1, -0.1, -0.1, None],
    [None, None, None, None, None, None, None]
]

In [ ]:
terminals = [(len(gridXD)-2, len(gridXD)-2)]

We define our maze environment below

In [ ]:
maze = GridMDP(gridXD, terminals)

To solve the maze, we can use the `best_policy` function along with `value_iteration`.

In [ ]:
maze

In [ ]:
uu = value_iteration(maze)

In [ ]:
uu

{(5, 1): 0.16208115706883838,
 (1, 3): 1.6740473762040475,
 (3, 3): 1.0343790365962378,
 (5, 5): 5.0,
 (4, 5): 4.2682926829268295,
 (3, 1): 0.16208115706900714,
 (2, 1): 0.12059623985102505,
 (1, 4): 2.090082298035416,
 (1, 5): 2.5192603951702575,
 (2, 3): 1.316932007524478,
 (4, 3): 0.7626891689382147,
 (2, 2): 1.0343790365962378,
 (5, 3): 0.5477243211569478,
 (4, 2): 0.5477243211569478,
 (2, 5): 3.0616938233629822,
 (4, 1): 0.3235224367984536,
 (1, 1): -5.991793935750375,
 (3, 5): 3.625817965496729}

In [ ]:
maze.to_grid(uu)

[[None, None, None, None, None, None, None],
 [None, 2.52, 3.06, 3.63, 4.27, 5.0, None],
 [None, 2.09, None, None, None, None, None],
 [None, 1.67, 1.32, 1.03, 0.76, 0.55, None],
 [None, None, 1.03, None, 0.55, None, None],
 [None, -5.99, 0.12, 0.16, 0.32, 0.16, None],
 [None, None, None, None, None, None, None]]

In [ ]:
pi = best_policy(maze, uu)

In [ ]:
pi

{(5, 1): (-1, 0),
 (1, 3): (0, 1),
 (3, 3): (-1, 0),
 (5, 5): None,
 (4, 5): (1, 0),
 (3, 1): (1, 0),
 (2, 1): (1, 0),
 (1, 4): (0, 1),
 (1, 5): (1, 0),
 (2, 3): (-1, 0),
 (4, 3): (-1, 0),
 (2, 2): (0, 1),
 (5, 3): (-1, 0),
 (4, 2): (0, 1),
 (2, 5): (1, 0),
 (4, 1): (0, 1),
 (1, 1): (1, 0),
 (3, 5): (1, 0)}

In [ ]:
print_table(maze.to_arrows(pi))

None   None   None   None   None   None   None
None   >      >      >      >      .      None
None   ^      None   None   None   None   None
None   ^      <      <      <      <      None
None   None   ^      None   ^      None   None
None   >      >      >      ^      <      None
None   None   None   None   None   None   None


As you can infer, we can find the path to the terminal state starting from any given state using this policy.
All maze problems can be solved by formulating it as a MDP.

Con el programa Grid MDP, se puede construir manualmente el entorno, para resolverlo gráficamente:

<img src='https://raw.githubusercontent.com/iapucp/IA-Diplomado/master/Inteligencia%20Artificial%20para%20Juegos/Codigo_Sesion6/gridMDP_values_manual.png' width=500px>

Ejecutando el programa auxiliar Grid MDP, se genera el siguiente heatmap:

<img src='https://raw.githubusercontent.com/iapucp/IA-Diplomado/master/Inteligencia%20Artificial%20para%20Juegos/Codigo_Sesion6/gridMDP_colors.png' width=500px>

Luego de resolver el entorno, se consiguen los siguientes valores:

<img src='https://raw.githubusercontent.com/iapucp/IA-Diplomado/master/Inteligencia%20Artificial%20para%20Juegos/Codigo_Sesion6/gridMDP_value_tables.png' width=200px>

